In [79]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import glob
!pip install openpyxl

day_folder = "/content/drive/My Drive/31 mei/"
file_paths = glob.glob(day_folder + "*.csv")

school_id = 44
class_id = 104

dataframes = [pd.read_csv(file, sep=';') for file in file_paths]
keyfile_general = pd.read_csv("Keyfile_csv.csv")
keyfile_specific = pd.read_excel("keyfile school 1 class 104.xlsx")


file_to_subject = {}
teacher_tag = '0x24046130BA41'


for file in file_paths:
    df_temp = pd.read_csv(file, sep=';')
    tag_id = df_temp['TagId'].iloc[0]  # This is the TrackLabID
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    try:

        subject_id = keyfile_specific.loc[keyfile_specific['tagnummer '] == tagnr, 'subject ID'].values[0] #added a space
    except:
        print("Tag not found:" + tag_id)
        print(tagnr)
        subject_id = None

    # Store TrackLabID (tag_id), Tagnumber, and subject ID in the dictionary
    file_to_subject[tag_id] = {'Tagnumber': tagnr, 'subject ID': subject_id}
#print(file_to_subject)

# Filter dataframes to remove unfound tags (except teacher)
filtered_dataframes = [
    df for df in dataframes
    if (file_to_subject.get(df['TagId'].iloc[0]) and
        file_to_subject[df['TagId'].iloc[0]]['subject ID'] is not None) or
       df['TagId'].iloc[0] == teacher_tag
]

dataframes = filtered_dataframes  # Replace original dataframes with filtered ones


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tag not found:0x24046130B9B6
26
Tag not found:0x24046130BA41
35
Tag not found:0x24046130C90F
34
Tag not found:0x24046130C8AB
9
Tag not found:0x24046130CCF9
20
Tag not found:0x24046130BDD0
31
Tag not found:0x24046130B6FA
22
Tag not found:0x24025F44B7A5
32
Tag not found:0x24046131EF0E
13


In [80]:


def process_timestamps(df):
    # Extract 'TrackLabID'
    tag_id = df['TagId'].iloc[0]

    try:
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f').dt.floor('s')
    except ValueError:
        # Handle cases where 'TimeStamp' is already in datetime format
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.floor('s')

    # Keep only the first entry for each second
    df = df.groupby('TimeStamp').first().reset_index()

    for column in ['X', 'Y', 'Z']:
        # Iteratively detects and replaces outliers until no more are found
        while True:
            # Calculates the difference between consecutive values
            diff = df[column].diff()

            # Identify outliers where the difference is greater than 1.5
            outlier_indices = diff[abs(diff) > 1.5].index

            # loop break condition
            if len(outlier_indices) == 0:
                break

            # Replace outliers with the prior value
            for index in outlier_indices:
                if index > 0:  # >0 avoids first row (which has no prior value)
                    df.loc[index, column] = df.loc[index - 1, column]

    # Convert 'TimeStamp' to datetime in all dataframes
    for df_ in dataframes:
        df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime

    time_range = pd.date_range(
        start=min(df_['TimeStamp'].min() for df_ in dataframes),
        end=max(df_['TimeStamp'].max() for df_ in dataframes),
        freq='s'
    )
    full_df = pd.DataFrame({'TimeStamp': time_range})

    # fills missing seconds
    df = pd.merge(full_df, df, left_on='TimeStamp', right_on='TimeStamp', how='left')

    # Forward fill and then backward fill to handle NaNs at the beginning and end
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].ffill().bfill()
    # Interpolate any remaining NaNs
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].interpolate(method='linear')

    # Add 'TagID' column back to the DataFrame
    df['TagId'] = tag_id
    return df

dataframes = [df for df in [process_timestamps(df) for df in dataframes] if df is not None]



<ipython-input-80-b8737840830b>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-80-b8737840830b>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-80-b8737840830b>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df

In [81]:
import pandas as pd

def find_extremes_and_context(df, column):
    """
    Manual check as outlier detection, was used to determine 1.5 as sensible value for allowed value jumps.
    """

    # Find index of minimum and maximum values
    min_index = df[column].idxmin()
    max_index = df[column].idxmax()

    # Print +-5 values around min/max
    print(f"Minimum value of {column}:")
    print(df.loc[max(0, min_index - 5):min(len(df), min_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])

    print(f"\nMaximum value of {column}:")
    print(df.loc[max(0, max_index - 5):min(len(df), max_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])


for df in dataframes:
    for column in ['X', 'Y']:
        find_extremes_and_context(df, column)

Minimum value of X:
               TimeStamp         X         Y         Z
3417 2023-05-31 09:28:27  6.854773  3.601073  0.263203
3418 2023-05-31 09:28:28  5.819774  3.601073  0.190111
3419 2023-05-31 09:28:29  4.587498  3.601073  0.234675
3420 2023-05-31 09:28:30  3.710245  3.601073  0.743395
3421 2023-05-31 09:28:31  2.925084  3.601073  0.498202
3422 2023-05-31 09:28:32  2.741637  3.601073  0.398074
3423 2023-05-31 09:28:33  2.741637  3.601073  0.424381
3424 2023-05-31 09:28:34  2.741637  3.601073  0.489081
3425 2023-05-31 09:28:35  2.741637  3.601073  0.415014
3426 2023-05-31 09:28:36  3.759116  3.601073  0.401398
3427 2023-05-31 09:28:37  5.255438  3.601073  0.608073

Maximum value of X:
              TimeStamp         X        Y         Z
797 2023-05-31 08:44:47  8.618712  1.94188  0.630641
798 2023-05-31 08:44:48  8.630636  1.94188  0.252994
799 2023-05-31 08:44:49  8.630636  1.94188  0.147560
800 2023-05-31 08:44:50  8.630636  1.94188  0.515242
801 2023-05-31 08:44:51  8.767147 

In [83]:
#don't actually use the averages, could be used for plotting though (I removed those cells from the notebook because we don't actually need it, could add back if you want)
object_averages = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    if tag_id == teacher_tag:
        tagnr = '35'
        subject_id = "Teacher"

    elif tag_id in df['TrackLabID'].values:
        tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
        subject_id = keyfile_specific.loc[keyfile_specific['tagnummer '] == tagnr, 'subject ID'].values[0]




    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].mean()
    object_averages[f"Person_{subject_id} with Tagnr {tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(object_averages, orient="index", columns=["X", "Y", "Z"])
print(avg_df)

                                     X         Y         Z
Person_Teacher with Tagnr 35  5.594560  3.469100  0.377162
Person_9 with Tagnr 7         2.467163  5.013641  0.315181
Person_10 with Tagnr 1        6.999096  5.444916  0.403120
Person_13 with Tagnr 11       4.601579  1.749246  0.404096
Person_1 with Tagnr 25        3.788207  4.745384  0.314283
Person_6 with Tagnr 15        2.817715  2.714488  0.393377
Person_16 with Tagnr 21       6.849180  4.472673  0.435652
Person_7 with Tagnr 18        2.085372  4.155170  0.407435
Person_3 with Tagnr 5         5.717306  2.707544  0.359224
Person_11 with Tagnr 17       3.963254  4.650267  0.339207
Person_4 with Tagnr 24        3.246754  4.556415  0.368089
Person_15 with Tagnr 6        5.363948  5.740222  0.393075
Person_12 with Tagnr 10       7.162728  1.683360  0.386165
Person_5 with Tagnr 27        6.277638  4.520692  0.473174


In [84]:
fivemin_object_averages = {}
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # Calculate average for the first 300 rows (= 5 minutes)
    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].head(300).mean()
    fivemin_object_averages[f"{tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(fivemin_object_averages, orient="index", columns=["X", "Y", "Z"])

print(avg_df)

           X         Y         Z
35  7.788556  1.470872  0.574337
7   7.752189  7.630444  0.231061
1   6.732990  5.315803  0.247188
11  3.570074  1.438972  0.276695
25  3.910266  3.630396  0.266675
15  2.872357  2.703514  0.410054
21  7.273767  3.982799  0.489947
18  2.758855  4.460601  0.282521
5   7.148278  2.535016  0.334721
17  4.090634  4.461383  0.216962
24  7.418781  3.916054  0.220162
6   6.998329  7.036130  0.201576
10  7.343309  2.517808  0.189212
27  6.217413  3.917685  0.331509


In [85]:
#this initially was for baseline distance purposes, decided using avg first 5 minutes might be better though due to trackers potentially not being turned on in usual positions (e.g. turning it on whilst picking it up from teachers desk)
object_start = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # respective 1st coordinates
    start_x = df_['X'].iloc[0]
    start_y = df_['Y'].iloc[0]
    start_z = df_['Z'].iloc[0]
    object_start[f"{tagnr}"] = (start_x, start_y, start_z)

start_positions_df = pd.DataFrame.from_dict(object_start, orient="index", columns=["X", "Y", "Z"])

print(start_positions_df)

           X         Y         Z
35  7.788556  1.470872  0.655372
7   7.752189  7.630444  0.231061
1   6.859102  5.435821  0.330802
11  3.540067  1.459106  0.247041
25  4.124514  3.515680  0.125077
15  2.730079  2.531990  0.318661
21  7.273767  3.982799  0.461040
18  2.146972  4.429597  0.785420
5   7.082801  2.558483  0.215321
17  4.101560  4.546058  0.247354
24  7.523614  3.984922  0.224117
6   6.998329  7.036130  0.201576
10  7.343309  2.517808  0.254356
27  6.217413  3.917685  0.164892


In [86]:
import numpy as np

def euclidean_distance(x1, y1, x2, y2):
  return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

target_tagnr = 27

# Find target coordinates using tagnr
target_coords = None
for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker == f"{target_tagnr}":
    target_coords = (x, y)
    break




target_x, target_y = target_coords
min_distance = float('inf')
closest_tracker = None

for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker != f"{target_tagnr}":  # Excludes teacher who would have distance 0
    distance = euclidean_distance(target_x, target_y, x, y)
    print(distance, tracker) #can be removed later
    if distance < min_distance:
      min_distance = distance
      closest_tracker = tracker

closest_distance_to_35 = min_distance
print(f"The closest tracker to the teacher is {closest_tracker} with a distance of {closest_distance_to_35}")

2.9078141286362333 35
4.017476002854635 7
1.4901517035144294 1
3.6266264517484745 11
2.3249645488468325 25
3.5585969656287633 15
1.0583593666591904 21
3.5009117916763044 18
1.666818300177036 5
2.195175389375602 17
1.2013696074030766 24
3.2147364768863054 6
1.7964686237822343 10
The closest tracker to the teacher is 21 with a distance of 1.0583593666591904


In [87]:
tracker_counts = {}
total_teacher_proximity_count = 0
teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher
print(len(teacher_df))
for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Initialise count for tracker
    tracker_counts[f"{tagnr}"] = 0


    # Iterate through rows using the index
    for index in range(len(df)):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < closest_distance_to_35:
            tracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
sorted_tracker_counts = dict(sorted(tracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within full distance of anyone: {total_teacher_proximity_count}")


exp_amount = 100 / (len(dataframes)-1) #determines expected proximity percentage if it were fully equal(e.g. 20 students = 5%), -1 due to not counting teacher
lower = 0.75
upper = 1.25
for tracker, count in sorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within full distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

12667
Total times teacher was within full distance of anyone: 23583
Person None with Tracker 35: 12667 times within full distance of teacher, category: above average with proximity percentage 53.71
Person 3 with Tracker 5: 9436 times within full distance of teacher, category: above average with proximity percentage 40.01
Person 9 with Tracker 7: 759 times within full distance of teacher, category: below average with proximity percentage 3.22
Person 16 with Tracker 21: 559 times within full distance of teacher, category: below average with proximity percentage 2.37
Person 12 with Tracker 10: 76 times within full distance of teacher, category: below average with proximity percentage 0.32
Person 6 with Tracker 15: 55 times within full distance of teacher, category: below average with proximity percentage 0.23
Person 4 with Tracker 24: 16 times within full distance of teacher, category: below average with proximity percentage 0.07
Person 7 with Tracker 18: 15 times within full distance of 

In [88]:
half_distance = closest_distance_to_35 / 2
htracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    htracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < half_distance:
            htracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
hsorted_tracker_counts = dict(sorted(htracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within half distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in hsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within half distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within half distance of anyone: 272
Person 3 with Tracker 5: 262 times within half distance of teacher, category: above average with proximity percentage 96.32
Person 12 with Tracker 10: 6 times within half distance of teacher, category: below average with proximity percentage 2.21
Person 9 with Tracker 7: 2 times within half distance of teacher, category: below average with proximity percentage 0.74
Person 16 with Tracker 21: 2 times within half distance of teacher, category: below average with proximity percentage 0.74
Person 10 with Tracker 1: 0 times within half distance of teacher, category: below average with proximity percentage 0.00
Person 13 with Tracker 11: 0 times within half distance of teacher, category: below average with proximity percentage 0.00
Person 1 with Tracker 25: 0 times within half distance of teacher, category: below average with proximity percentage 0.00
Person 6 with Tracker 15: 0 times within half distance of teacher, category: below

In [89]:
quarter_distance = closest_distance_to_35/4

qtracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    qtracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < quarter_distance:
            qtracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
qsorted_tracker_counts = dict(sorted(qtracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within quarter distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in qsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within quarter distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within quarter distance of anyone: 10
Person 3 with Tracker 5: 10 times within quarter distance of teacher, category: above average with proximity percentage 100.00
Person 9 with Tracker 7: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 10 with Tracker 1: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 13 with Tracker 11: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 1 with Tracker 25: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 6 with Tracker 15: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 16 with Tracker 21: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 7 with Tracker 18: 0 times within quarter distance of

In [90]:

merge_df = pd.read_excel("merged-data-WIP_2025-05-01_17-31 (1) (1) (1).xlsx")  # Replace with your file name

# Create new columns if they don't exist
for distance_type in ['full', 'half', 'quarter']:
    if f'{distance_type}_proximity' not in merge_df.columns:
        merge_df[f'{distance_type}_proximity'] = None
    if f'{distance_type}_category' not in merge_df.columns:
        merge_df[f'{distance_type}_category'] = None


for distance_type, sorted_counts in [('full', sorted_tracker_counts), ('half', hsorted_tracker_counts), ('quarter', qsorted_tracker_counts)]:
    total_count = sum(sorted_counts.values()) # total proximity count for this distance type

    for tracker, count in sorted_counts.items():

        tag_id = df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0]

        # Find the matching row in the existing DataFrame
        mask = (merge_df['school'] == school_id) & (merge_df['class'] == class_id) & (merge_df['tracklab_id'] == tag_id)  # Replace 'tracklab_id' with the actual column name for TagID if different
        matching_row_index = merge_df[mask].index

        # Update the existing DataFrame if matching row is found
        if len(matching_row_index) > 0:
            row_index = matching_row_index[0]
            merge_df.loc[row_index, f'{distance_type}_proximity'] = (count / total_count) * 100

            proximity_percentage = (count / total_count) * 100
            ratio = proximity_percentage / exp_amount

            if ratio < lower:
                category = "below average"
            elif ratio > upper:
                category = "above average"
            else:
                category = "average"

            merge_df.loc[row_index, f'{distance_type}_category'] = category


merge_df.to_excel("merged-data-WIP_2025-05-01_17-31 (1) (1) (1).xlsx", index=False)